<a href="https://colab.research.google.com/github/Xavier-ML/What-s-in-an-Avocado-Toast-A-Supply-Chain-Analysis/blob/main/What's_in_an_Avocado_Toast_A_Supply_Chain_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What's in an Avocado Toast: A Supply Chain Analysis

![](avocado_wallpaper.jpeg)

You find yourself in London, crafting a delectable avocado toast, a dish that has risen dramatically in popularity on breakfast menus since the 2010s. This straightforward recipe requires just five ingredients: a ripe avocado, half a lemon, a generous pinch of salt flakes, two slices of sourdough bread, and a good drizzle of extra virgin olive oil. Most of these ingredients are now staples in grocery stores, and as you will find with this project, that is no small feat!

In this project, you'll conduct a supply chain analysis of three ingredients used in avocado toast using the Open Food Facts database. This database contains extensive, openly-sourced information on various foods, including their origins. Through this analysis, you will gain an in-depth understanding of the complex supply chain involved in producing a single dish.

Three pairs of files are provided in the data folder:
- A CSV file for each ingredient, such as `avocado.csv`, with data about each food item and countries of origin.
- A TXT file for each ingredient, such as `relevant_avocado_categories`, containing only the category tags of interest for that food.

Here are some other key points about these files:
- Some of the rows of data in each of the three CSV files do not contain relevant data for your investigation. In each dataset, you will need to filter out rows with irrelevant data, based on values in the `categories_tags` column. Examples of categories are fruits, vegetables, and fruit-based oils. Filter the DataFrame to include only rows where `categories_tags` contains one of the tags in the relevant categories for that ingredient.
- Each row of data usually has multiple category tags in the `categories_tags` column.
There is a column in each CSV file called `origins_tags`, which contains strings for the country of origin of each item.

After completing this project, you'll be armed with a list of ingredients and their countries of origin and be well-positioned to launch into other analyses that explore how long, on average, these ingredients spend at sea.

[Open Food Facts database](https://world.openfoodfacts.org/)

In [ ]:
import pandas as pd

In [ ]:
avocado = pd.read_csv('data/avocado.csv', sep='\t')
avocado.head()

,code,lc,product_name_de,product_name_el,product_name_en,product_name_es,product_name_fi,product_name_fr,product_name_id,product_name_it,product_name_lt,product_name_lv,product_name_nb,product_name_nl,product_name_pl,product_name_ro,product_name_sl,product_name_sv,generic_name_de,generic_name_en,generic_name_es,generic_name_fr,generic_name_sv,quantity,serving_size,packaging,packaging_tags,brands,brands_tags,brand_owner,categories,categories_tags,labels,labels_tags,countries,countries_tags,stores,stores_tags,obsolete,obsolete_since_date,...,phosphorus_unit,iron_value,iron_unit,magnesium_value,magnesium_unit,zinc_value,zinc_unit,copper_value,copper_unit,manganese_value,manganese_unit,selenium_value,selenium_unit,fruits-vegetables-nuts_value,fruits-vegetables-nuts_unit,phylloquinone_value,phylloquinone_unit,link,off:food_groups,off:food_groups_tags,off:nova_groups,off:nova_groups_tags,off:nutriscore_grade,off:nutriscore_score,off:ecoscore_grade,off:ecoscore_score,off:ecoscore_data.missing_key_data,off:ecoscore_data.agribalyse.code,off:ecoscore_data.adjustments.origins_of_ingredients.value,off:ecoscore_data.adjustments.packaging.value,off:ecoscore_data.adjustments.packaging.non_recyclable_and_non_biodegradable_materials,off:ecoscore_data.adjustments.production_system.value,off:ecoscore_data.adjustments.threatened_species.value,sources_fields:org-database-usda:available_date,sources_fields:org-database-usda:fdc_category,sources_fields:org-database-usda:fdc_data_source,sources_fields:org-database-usda:fdc_id,sources_fields:org-database-usda:modified_date,sources_fields:org-database-usda:publication_date,data_sources
0,0059749979702,fr,NaN,NaN,NaN,NaN,NaN,Naturalia Avocado Oil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Naturalia,naturalia,NaN,"Aliments et boissons à base de végétaux, Alime...","en:plant-based-foods-and-beverages,en:plant-ba...",NaN,NaN,Canada,en:canada,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:fats,"en:fats-and-sauces,en:fats",2.0,en:2-processed-culinary-ingredients,b,1.0,unknown,NaN,1.0,NaN,-5.0,-15.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"App - yuka, Apps"
1,7610095131409,en,NaN,NaN,NaN,NaN,NaN,Avocado Bowl chips,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zweifel,zweifel,NaN,"Snacks, Salty snacks, Appetizers, Chips and fr...","en:snacks,en:salty-snacks,en:appetizers,en:chi...","Vegetarian, Vegan","en:vegetarian,en:vegan","Switzerland, World","en:switzerland,en:world",NaN,NaN,1,2023/03/21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:appetizers,"en:salty-snacks,en:appetizers",NaN,unknown,c,8.0,unknown,NaN,1.0,NaN,-5.0,-15.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"App - Yuka, Apps, Producers, Producer - zweifel"
2,4005514005578,en,NaN,NaN,Gelbe Linse Avocado Brotaufstrich,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tartex,tartex,NaN,de:Abendbrotsufstrich,de:abendbrotsufstrich,"Organic, EU Organic, EG-Öko-Verordnung","en:organic,en:eu-organic,en:eg-oko-verordnung",Germany,en:germany,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,b,0.0,unknown,NaN,1.0,NaN,-5.0,-15.0,1.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"App - yuka, Apps, App - smoothie-openfoodfacts"
3,0879890002513,en,NaN,NaN,Avocado toast chili lime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,en:4-ultra-processed-food-and-drink-products,NaN,NaN,unknown,NaN,1.0,NaN,-5.0,-15.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"App - Yuka, Apps, App - InFood"
4,0223086613685,en,NaN,NaN,Avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-stat

In [ ]:
relevant_avocado_categories_csv = pd.read_csv('data/relevant_avocado_categories.txt', sep='\t')
relevant_avocado_categories_csv

,en:avocadoes
0,en:avocados
1,en:fresh-foods
2,en:fresh-vegetables
3,en:fruchte
4,en:fruits
5,en:raw-green-avocados
6,en:tropical-fruits
7,en:tropische-fruchte
8,en:vegetables-based-foods
9,fr:hass-avocados


In [ ]:
with open('data/relevant_avocado_categories.txt', "r") as file:
    relevant_avocado_categories = file.read().splitlines()
    file.close()

relevant_avocado_categories

['en:avocadoes',
 'en:avocados',
 'en:fresh-foods',
 'en:fresh-vegetables',
 'en:fruchte',
 'en:fruits',
 'en:raw-green-avocados',
 'en:tropical-fruits',
 'en:tropische-fruchte',
 'en:vegetables-based-foods',
 'fr:hass-avocados']

In [ ]:
# Subset the avocado DataFrame to include only the relevant columns
relevant_columns = ['code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands', 'brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins', 'origins_tags']
avocado_subset = avocado[relevant_columns]
avocado_subset.head()

,code,lc,product_name_en,quantity,serving_size,packaging_tags,brands,brands_tags,categories_tags,labels_tags,countries,countries_tags,origins,origins_tags
0,0059749979702,fr,NaN,NaN,NaN,NaN,Naturalia,naturalia,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,Canada,en:canada,NaN,NaN
1,7610095131409,en,NaN,NaN,NaN,NaN,Zweifel,zweifel,"en:snacks,en:salty-snacks,en:appetizers,en:chi...","en:vegetarian,en:vegan","Switzerland, World","en:switzerland,en:world",NaN,NaN
2,4005514005578,en,Gelbe Linse Avocado Brotaufstrich,NaN,NaN,NaN,Tartex,tartex,de:abendbrotsufstrich,"en:organic,en:eu-organic,en:eg-oko-verordnung",Germany,en:germany,NaN,NaN
3,0879890002513,en,Avocado toast chili lime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,NaN,NaN
4,0223086613685,en,Avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,NaN,NaN


In [ ]:
# Drop rows with null values in 'categories_tags'
avocado_subset = avocado_subset.dropna(subset=['categories_tags'])

# Ensure 'categories_tags' is a string before splitting
avocado_subset['categories_tags'] = avocado_subset['categories_tags'].apply(lambda x: x if isinstance(x, list) else x.split(','))

# Filter the DataFrame to contain only rows with relevant category tags
filtered_avocado = avocado_subset[avocado_subset['categories_tags'].apply(lambda tags: any(tag in relevant_avocado_categories for tag in tags))]

filtered_avocado.head()

,code,lc,product_name_en,quantity,serving_size,packaging_tags,brands,brands_tags,categories_tags,labels_tags,countries,countries_tags,origins,origins_tags
5,3662994002063,fr,NaN,3 fruits,NaN,NaN,la compagnie des fruits mûrs,la-compagnie-des-fruits-murs,"[en:plant-based-foods-and-beverages, en:plant-...",NaN,France,en:france,NaN,NaN
6,8437013031011,fr,NaN,1 kg,NaN,NaN,NaN,NaN,"[en:plant-based-foods-and-beverages, en:plant-...",NaN,France,en:france,NaN,NaN
14,4016249238155,de,NaN,135g,100g,de:gläschen,Allos,allos,"[en:plant-based-foods-and-beverages, en:plant-...","en:organic,en:vegetarian,en:eu-organic,en:no-g...",Deutschland,en:germany,Europäische Union,en:european-union
17,8718963381532,de,NaN,NaN,NaN,NaN,NaN,NaN,"[en:plant-based-foods-and-beverages, en:plant-...",NaN,Deutschland,en:germany,NaN,NaN
23,8436002746707,es,NaN,NaN,NaN,NaN,NaN,NaN,"[en:plant-based-foods-and-beverages, en:plant-...",NaN,España,en:spain,NaN,NaN


In [ ]:
# Filter the DataFrame to include only avocados from the United Kingdom
avocados_uk = filtered_avocado[filtered_avocado['countries'] == 'United Kingdom']

# Get the most common origin of avocados in the UK
top_avocado_origin = avocados_uk['origins_tags'].value_counts().index[0]

# Strip out characters before the country name and replace hyphens with spaces
top_avocado_origin = top_avocado_origin.lstrip("en:").replace('-', ' ')

top_avocado_origin

'peru'

In [ ]:
def read_and_filter_data(filename, relevant_categories):
  df = pd.read_csv('data/' + filename, sep='\t')

  # Subset large DataFrame to include only relevant columns
  subset_columns = [ 'code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands', 'brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins','origins_tags']
  df = df[subset_columns]

  # Split tags into lists
  df['categories_list'] = df['categories_tags'].str.split(',')

  # Drop rows with null categories data
  df = df.dropna(subset = 'categories_list')

  # Filter data for relevant categories
  df = df[df['categories_list'].apply(lambda x: any([i for i in x if i in relevant_categories]))]

  # Filter data for the UK
  df_uk = df[(df['countries']=='United Kingdom')]

  # Find top origin country string with the highest count
  top_origin_string = (df_uk['origins_tags'].value_counts().index[0])

  # Clean up top origin country string
  top_origin_country = top_origin_string.lstrip("en:")
  top_origin_country = top_origin_country.replace('-', ' ')

  print(f'**{filename[:-4]} origins**','\n', top_origin_country, '\n')

  print ("Top origin country: ", top_origin_country)
  print ("\n")

  # End of function - return top origin country for this ingredient
  return top_origin_country


# Analyze avocado origins again, this time by calling function
top_avocado_origin = read_and_filter_data('avocado.csv',relevant_avocado_categories)

### Repeat process above with new function for the other 2 ingredients

# Gather relevant categories data for olive oil
with open("data/relevant_olive_oil_categories.txt", "r") as file:
    relevant_olive_oil_categories = file.read().splitlines()
    file.close()

# Call user-defined function on olive_oil.csv
top_olive_oil_origin = read_and_filter_data('olive_oil.csv',relevant_olive_oil_categories)

# Gather relevant categories data for sourdough
with open("data/relevant_sourdough_categories.txt", "r") as file:
    relevant_sourdough_categories = file.read().splitlines()
    file.close()

# Call user-defined function on sourdough.csv
top_sourdough_origin = read_and_filter_data('sourdough.csv',relevant_sourdough_categories)

**avocado origins** 
 peru 

Top origin country:  peru


**olive_oil origins** 
 greece 

Top origin country:  greece


**sourdough origins** 
 united kingdom 

Top origin country:  united kingdom


